In [1]:
# Creating an experiment with your api key
from comet_ml import Experiment
experiment = Experiment(
    api_key = "XTvxi4t0C4xz1pFbi15s0rvsQ",
    project_name = "classification",
    workspace = "bolamash"
    )

COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Couldn't find a Git repository in "C:\\Users\\Ipaye Bolanle Mas'ud\\Documents\\Classification\\predict" nor in any parent directory. Set `COMET_GIT_DIRECTORY` if your Git Repository is elsewhere.
COMET INFO: Experiment is live on comet.com https://www.comet.com/bolamash/classification/e11bc8cc59cb4bce999838bd8b0ea50f



In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib.colors import ListedColormap

from sklearn.feature_extraction.text import CountVectorizer

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn import metrics

from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_moons, make_circles, make_classification

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier

# Libraries for data loading, manipulation and visualisation
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.style as style
from wordcloud import WordCloud
from plotly import graph_objects as go 
sns.set()
%matplotlib inline

# Libraries for data preparation and model building
from sklearn.utils import resample
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC, SVC
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression, SGDClassifier, RidgeClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import GridSearchCV, StratifiedKFold, KFold
from sklearn.pipeline import Pipeline
from sklearn import metrics
from sklearn.model_selection import cross_val_score, RepeatedStratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, recall_score, precision_score, f1_score

#nlkt libraries
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import nltk
import re
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')


import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package punkt to C:\Users\Ipaye Bolanle
[nltk_data]     Mas'ud\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\Ipaye Bolanle
[nltk_data]     Mas'ud\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\Ipaye Bolanle
[nltk_data]     Mas'ud\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
train_df = pd.read_csv('train_set.csv')
train_df.head()

,lang_id,text
0,xho,umgaqo-siseko wenza amalungiselelo kumaziko ax...
1,xho,i-dha iya kuba nobulumko bokubeka umsebenzi na...
2,eng,the province of kwazulu-natal department of tr...
3,nso,o netefatša gore o ba file dilo ka moka tše le...
4,ven,khomishini ya ndinganyiso ya mbeu yo ewa maana...


In [4]:
test_df = pd.read_csv('test_set.csv')
test_df.head()

,index,text
0,1,"Mmasepala, fa maemo a a kgethegileng a letlele..."
1,2,Uzakwaziswa ngokufaneleko nakungafuneka eminye...
2,3,Tshivhumbeo tshi fana na ngano dza vhathu.
3,4,Kube inja nelikati betingevakala kutsi titsini...
4,5,Winste op buitelandse valuta.


In [5]:
sample_df = pd.read_csv('sample_submission.csv')
sample_df.head()

,index,lang_id
0,1,tsn
1,2,nbl


In [6]:
print('Train Dataset')
display(train_df.isnull().sum())

print('Test Dataset')
display(test_df.isnull().sum())

Train Dataset


lang_id    0
text       0
dtype: int64

Test Dataset


index    0
text     0
dtype: int64

In [7]:
#Check shape of datasets
print(f'The shape of the train dataset: {train_df.shape}')
print(f'The shape of the test dataset:  {test_df.shape}')

The shape of the train dataset: (33000, 2)
The shape of the test dataset:  (5682, 2)


In [8]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33000 entries, 0 to 32999
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   lang_id  33000 non-null  object
 1   text     33000 non-null  object
dtypes: object(2)
memory usage: 515.8+ KB


In [9]:
# check the unique values in target feature
train_df['lang_id'].unique()

array(['xho', 'eng', 'nso', 'ven', 'tsn', 'nbl', 'zul', 'ssw', 'tso',
       'sot', 'afr'], dtype=object)

### lang_id description

afr - Afrikaans

eng - English

nbl - isiNdebele

nso - Sepedi

sot - Sesotho

ssw - siSwati

tsn - Setswana

tso - Xitsonga

ven - Tshivenda

xho - isiXhosa

zul - isiZulu

In [10]:
# Select all duplicate rows based on the message column from train dataset
duplicate_train_df = train_df[train_df.
                    duplicated(['text',
                                'lang_id'],
                               keep=False)]

# View number of duplicate rows
duplicate_train_df.shape

(5599, 2)

In [11]:
#function to convert class from numerical to word definition of class for a better visualisation
def class_convert(df):
    df = train_df.copy()
    word_class = []
    old_class = df['lang_id']
    
    for class_id in old_class:
        if class_id  == 'zul':
            word_class.append('isizulu')
        elif class_id  == 'xho':
            word_class.append('isiXhosa')
        elif class_id  == 'ven':
            word_class.append('Tshivenda')
        elif class_id  == 'tso':
            word_class.append('Xitsonga')
        elif class_id  == 'tsn':
            word_class.append('Setswana')
        elif class_id == 'ssw':
            word_class.append('siSwati')
        elif class_id  == 'sot':
            word_class.append('Sesotho')
        elif class_id == 'nso':
            word_class.append('Sepedi')
        elif class_id == 'nbl':
            word_class.append('isiNdebele')
        elif class_id == 'eng' :
            word_class.append(' English')
        else:
            word_class.append('Afrikaans')
    df['lang_id'] = word_class
        
    return df

#saving a working copy of train dataset
train_copy = class_convert(train_df)
#saving a working copy of test dataset
test_copy = test_df.copy()

In [12]:
#Veiwing the new edited first 5 rows of the train dataframe
train_copy.head()

,lang_id,text
0,isiXhosa,umgaqo-siseko wenza amalungiselelo kumaziko ax...
1,isiXhosa,i-dha iya kuba nobulumko bokubeka umsebenzi na...
2,English,the province of kwazulu-natal department of tr...
3,Sepedi,o netefatša gore o ba file dilo ka moka tše le...
4,Tshivenda,khomishini ya ndinganyiso ya mbeu yo ewa maana...


In [13]:
#checking message count grouped by sentiment
sentiment_temp_table = train_copy.groupby('lang_id').count()['text'].reset_index().sort_values(by='text')
sentiment_temp_table

,lang_id,text
0,English,3000
1,Afrikaans,3000
2,Sepedi,3000
3,Sesotho,3000
4,Setswana,3000
5,Tshivenda,3000
6,Xitsonga,3000
7,isiNdebele,3000
8,isiXhosa,3000
9,isizulu,3000


### Data engineering

In [14]:
#function to replace all url links with the word 'url'
def replace_tweet_urls(df, column_name):
    
    # Describing regrex url pattern
    pattern_url = r'http[s]?://(?:[A-Za-z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9A-Fa-f][0-9A-Fa-f]))+'
    
    #substitute text to replace url
    sub_url = r'url'
    
    df[column_name] = df[column_name].replace(to_replace = pattern_url,
                            value = sub_url, 
                            regex = True) #train dataset
    return df

# Applying replace_tweet_urls function on both datasets replace all long urls with just 'url'  
replace_tweet_urls(train_copy, 'text')
replace_tweet_urls(test_copy, 'text')

,index,text
0,1,"Mmasepala, fa maemo a a kgethegileng a letlele..."
1,2,Uzakwaziswa ngokufaneleko nakungafuneka eminye...
2,3,Tshivhumbeo tshi fana na ngano dza vhathu.
3,4,Kube inja nelikati betingevakala kutsi titsini...
4,5,Winste op buitelandse valuta.
...,...,...
5677,5678,You mark your ballot in private.
5678,5679,Ge o ka kgetha ka bowena go se šomiše Mofani k...
5679,5680,"E Ka kopo etsa kgetho ya hao ka hloko, hobane ..."
5680,5681,"TB ke bokudi ba PMB, mme Morero o tla lefella ..."


In [15]:
#visualising the cleaned columns
train_copy.head()

,lang_id,text
0,isiXhosa,umgaqo-siseko wenza amalungiselelo kumaziko ax...
1,isiXhosa,i-dha iya kuba nobulumko bokubeka umsebenzi na...
2,English,the province of kwazulu-natal department of tr...
3,Sepedi,o netefatša gore o ba file dilo ka moka tše le...
4,Tshivenda,khomishini ya ndinganyiso ya mbeu yo ewa maana...


In [16]:
#Split Down sampled train dataset features into Predictors and Target 
X_sampled = train_copy['text']
y_sampled = train_copy['lang_id']


unseen_X = test_copy['text'] 

### Modelling and Model Performance

In [17]:
def model_trainer(model, X, y):
    
    
    #"""create train model function"""
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=20)
    
    pipe = Pipeline([('vect', TfidfVectorizer(stop_words='english', 
                             min_df=1, 
                             max_df=0.9, 
                             ngram_range=(1, 3))),('tfidf', TfidfTransformer()),('model', model)])
    
    
    
    pipe.fit(X_train, y_train)
    y_pred = pipe.predict(X_test)
    f1_mscore = f1_score(y_test, y_pred, average='weighted')
    accuracy_mscore = accuracy_score(y_test, y_pred)
    recall_mscore = recall_score(y_test, y_pred, average='weighted')
    precision_mscore = precision_score(y_test, y_pred, average='weighted')
        
       
    print('Accuracy: ', accuracy_mscore)
    print('f1_score: ', f1_mscore)
    print('Recall: ', recall_mscore)
    print('Precision: ', precision_mscore)
    
    metrics = {'Accuracy':accuracy_mscore, 'f1_score':f1_mscore, 'Recall': recall_mscore, 'Precision':precision_mscore}
    #experiment.log_metrics(metrics)
    #experiment.end()
    return f1_mscore, accuracy_mscore, recall_mscore, precision_mscore

### Metrics

Below, we will calculate the metrics for each model.

1. Logistic Regression
Logistic regression is a classification algorithm used to predict the probability of an event occurring or belonging to a specific class. It's commonly used when the outcome variable is binary or categorical.

In [18]:
#Creating an object of Logistic Regression
log_regression = LogisticRegression(multi_class='ovr')

In [19]:
imbalanced_lr_f1, imbalanced_lr_accuracy, imbalanced_lr_recall,\
imbalanced_lr_presicion = model_trainer(log_regression, X_sampled, y_sampled)

Accuracy:  0.998030303030303
f1_score:  0.9980309474183174
Recall:  0.998030303030303
Precision:  0.9980361311061432


### 2. Decision Tree
Decision trees are versatile classification algorithms that make predictions based on a series of if-else decisions. They are easy to interpret and can handle both categorical and numerical data.

In [20]:
#Creating a decision tree classifier
tree = DecisionTreeClassifier(random_state=42)

In [21]:
imbalanced_tree_f1, imbalanced_tree_accuracy, imbalanced_tree_recall,\
imbalanced_tree_presicion = model_trainer(tree, X_sampled, y_sampled)

Accuracy:  0.9240909090909091
f1_score:  0.9231796767956224
Recall:  0.9240909090909091
Precision:  0.9240089441197742


### 3. Random Forest
Random forests are an ensemble method that combines multiple decision trees to make more accurate predictions. They reduce overfitting and improve generalization by averaging the predictions of multiple trees.

In [22]:
#Creating an object of Random Forest
forest = RandomForestClassifier(n_estimators=100, max_features=1)

In [23]:
imbalanced_forest_f1, imbalanced_forest_accuracy, imbalanced_forest_recall,\
imbalanced_forest_presicion = model_trainer(forest, X_sampled, y_sampled)

Accuracy:  0.8895454545454545
f1_score:  0.9000113055751495
Recall:  0.8895454545454545
Precision:  0.9501562375534319


### 4. Naive Bayes
Multinomial Naive Bayes is a classification algorithm based on Bayes' theorem. It's commonly used for text classification problems, such as sentiment analysis or spam detection.

In [24]:
#Creating an object of Naive Bayes
naive_bayes = MultinomialNB()

In [25]:
imbalanced_naiveb_f1, imbalanced_naiveb_accuracy, imbalanced_naiveb_recall,\
imbalanced_naiveb_presicion = model_trainer(naive_bayes, X_sampled, y_sampled)

Accuracy:  0.9995454545454545
f1_score:  0.9995455723429163
Recall:  0.9995454545454545
Precision:  0.9995466822443402


Best Performing Model
i have chosen the Multinomial Naive Bayes as my best performing model.

Multinomial Naive Bayes has been widely used and studied in various natural language processing tasks, including sentiment analysis. It has demonstrated good performance in terms of accuracy and robustness, making it a popular choice in many text classification scenarios.

### Predict Function and Submission

In [26]:
#create predict function
def test_target_predictor(model, X, y, test):
    
    pipe = Pipeline([('vect', TfidfVectorizer(stop_words='english', 
                             min_df=1, 
                             max_df=0.9, 
                             ngram_range=(1, 3))),
                     ('tfidf', TfidfTransformer()),
                     ('model', model)])
    
    pipe.fit(X, y)
    y_pred = pipe.predict(test)
    
    return y_pred

In [27]:
y_pred_mnb = test_target_predictor(naive_bayes, X_sampled, y_sampled, unseen_X)

In [31]:
y_pred_mnb

array(['Setswana', 'isiNdebele', 'Tshivenda', ..., 'Sesotho', 'Sesotho',
       'isiNdebele'], dtype='<U10')

In [36]:
#Creating the kaggle submission Dataframe
test = test_copy.set_index('index')
final_test = test.index


# Creating the Kaggle submission DataFrame without rounding
final_dataframe = {'index': final_test, 'lang_id': y_pred_mnb}
k_submission = pd.DataFrame(data=final_dataframe)
k_submission = k_submission[['index', 'lang_id']]

In [37]:
k_submission.to_csv('mnb.csv', index = False) #writing csv file

In [38]:
# To end all experiments for Comet
experiment.end()

COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.com/bolamash/classification/e11bc8cc59cb4bce999838bd8b0ea50f
COMET INFO:   Parameters:
COMET INFO:     C                               : 1.0
COMET INFO:     alpha                           : 1.0
COMET INFO:     bootstrap                       : True
COMET INFO:     ccp_alpha                       : 0.0
COMET INFO:     class_prior                     : 1
COMET INFO:     class_weight                    : 1
COMET INFO:     criterion                       : gini
COMET INFO:     dual                            : False
COMET INFO:     fit_intercept                   : True
COMET INFO:     fit_prior                       : True
COMET INFO:

COMET INFO:     vect__analyzer                  : word
COMET INFO:     vect__binary                    : False
COMET INFO:     vect__decode_error              : strict
COMET INFO:     vect__dtype                     : <class 'numpy.float64'>
COMET INFO:     vect__encoding                  : utf-8
COMET INFO:     vect__input                     : content
COMET INFO:     vect__lowercase                 : True
COMET INFO:     vect__max_df                    : 0.9
COMET INFO:     vect__max_features              : 1
COMET INFO:     vect__min_df                    : 1
COMET INFO:     vect__ngram_range               : (1, 3)
COMET INFO:     vect__norm                      : l2
COMET INFO:     vect__preprocessor              : 1
COMET INFO:     vect__smooth_idf                : True
COMET INFO:     vect__stop_words                : english
COMET INFO:     vect__strip_accents             : 1
COMET INFO:     vect__sublinear_tf              : False
COMET INFO:     vect__token_pattern             